# Time-Indexed Job-Shop

$$
\begin{align}
    \text{min} \quad & C \\
    \text{s.t.} \quad & \sum_{t \in T}{y_{m, j, t}} = 1
        & \forall ~ j \in J; m \in M \\
    & \sum_{j \in J} x_{m, j, t} \leq 1
        & \forall ~ m \in M; t \in T \\
    & \sum_{t \in T}{(t + p_{\sigma_{h - 1}^j, j}) y_{\sigma_{h - 1}^j, j, t}} \leq
    \sum_{t \in T}{t y_{\sigma_{h}^j, j, t}}
        & \forall ~ j \in J; h \in (1, 2, ..., |M|) \\
    & y_{m, j, t} \leq x_{m, j, t}
        & \forall ~ m \in M; j \in J; t \in T \\
    & y_{m, j, t} \leq 1 - x_{m, j, t - 1}
        & \forall ~ m \in M; j \in J; t \in T \setminus \{0\} \\
    & x_{m, j, t} - x_{m, j, t - 1} \leq y_{m, j, t}
        & \forall ~ m \in M; j \in J; t \in T \setminus \{0\} \\
    & t x_{m, j, t} \leq C
        & \forall ~ m \in M; j \in J; t \in T \\

    & y_{m, j, t}, x_{m, j, t} \in \{0, 1\} & \forall ~ j; m \in M; t \in T\\
\end{align}
$$

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

### Libraries

In [2]:
import json
import pathlib

import numpy as np
import pandas as pd 
import pyomo.environ as pyo

In [3]:
from plotly import express as px
from plotly import graph_objects as go

### Configuration

In [4]:
pd.options.plotting.backend = "plotly"

## Data

### Read

In [5]:
pathlib.Path.cwd()

WindowsPath('c:/Users/kslad/Documents/projects/optimization/projects/time_indexed')

In [6]:
data_directory = pathlib.Path("data")
data_directory.absolute()

WindowsPath('c:/Users/kslad/Documents/projects/optimization/projects/time_indexed/data')

In [7]:
with (data_directory / "random_3_4.json").open(mode="r") as file:
    data = json.load(file)

data

{'technology': [[0, 1, 2], [2, 0, 1], [1, 2, 0], [0, 1, 2]],
 'processing': [{'machine': 0, 'job': 0, 'time': 9},
  {'machine': 0, 'job': 1, 'time': 10},
  {'machine': 0, 'job': 2, 'time': 18},
  {'machine': 0, 'job': 3, 'time': 14},
  {'machine': 1, 'job': 0, 'time': 7},
  {'machine': 1, 'job': 1, 'time': 16},
  {'machine': 1, 'job': 2, 'time': 11},
  {'machine': 1, 'job': 3, 'time': 15},
  {'machine': 2, 'job': 0, 'time': 5},
  {'machine': 2, 'job': 1, 'time': 10},
  {'machine': 2, 'job': 2, 'time': 13},
  {'machine': 2, 'job': 3, 'time': 17}]}

### Prepare

In [8]:
machines = sorted(data["technology"][0])
jobs = list(range(len(data["technology"])))
processing_time = {
    (record["machine"], record["job"]): record["time"]
    for record in data["processing"]
}
sequences = [
    (m1, row[h + 1], j)
    for j, row in enumerate(data["technology"])
    for h, m1 in enumerate(row[:-1])
]

sequences

[(0, 1, 0),
 (1, 2, 0),
 (2, 0, 1),
 (0, 1, 1),
 (1, 2, 2),
 (2, 0, 2),
 (0, 1, 3),
 (1, 2, 3)]

## Model

In [9]:
model = pyo.ConcreteModel()

### Sets

In [10]:
model.add_component('machines', pyo.Set(initialize=machines))
model.add_component('jobs', pyo.Set(initialize=jobs))
model.add_component('time_periods', pyo.Set(initialize=range(sum(p for p in processing_time.values()))))
model.add_component('sequences', pyo.Set(initialize=sequences))

### Parameters


In [11]:
model.add_component('processing_time', pyo.Param(model.machines, model.jobs, initialize=processing_time))

### Decision Variables

In [12]:
model.add_component('x', pyo.Var(model.machines, model.jobs, model.time_periods, within=pyo.NonNegativeReals, bounds=(0, 1)))
model.add_component('y', pyo.Var(model.machines, model.jobs, model.time_periods, within=pyo.Binary))
model.add_component('C', pyo.Var(within=pyo.NonNegativeReals))

### Constraints

In [13]:
def constraint_unique_start(model, machine, job):
    return sum(model.y[machine, job, : ])  == 1


def constraint_unique_machine(model, machine, time_period):
    return sum(model.x[machine, :, time_period]) <= 1


def constraint_sequence_order(model, machine_1, machine_2, job):
    lhs = sum((time_period + model.processing_time[machine_1, job]) * model.y[machine_1, job, time_period] for time_period in model.time_periods)
    rhs = sum((time_period + model.processing_time[machine_2, job]) * model.y[machine_2, job, time_period] for time_period in model.time_periods)
    return lhs <= rhs


def constraint_job_duration(model, machine, job, time_period):
    if time_period <= model.time_periods.last() - model.processing_time[machine, job]:
        lhs = model.processing_time[machine, job] * model.y[machine, job, time_period]
        rhs = sum(model.x[machine, job, ending_time_period] for ending_time_period in range(time_period, time_period + model.processing_time[machine, job]))
        return lhs <= rhs
    else:
        return model.y[machine, job, time_period] == 0.0


def constraint_duration_complement(model, machine, job):
    return sum(model.x[machine, job, :]) == model.processing_time[machine, job]


def constraint_start_c1(model, machine, job, time_period):
    return model.y[machine, job, time_period] <= model.x[machine, job, time_period]


def constraint_start_c2(model, machine, job, time_period):
    if time_period != model.time_periods.first():
        previous_time_period = model.time_periods.prev(time_period)
        return model.y[machine, job, time_period] <= 1.0 - model.x[machine, job, previous_time_period]
    else:
        return pyo.Constraint.Skip


def constraint_start_c3(model, machine, job, time_period):
    if time_period != model.time_periods.first():
        previous_time_period = model.time_periods.prev(time_period)
        return model.x[machine, job, time_period] - model.x[machine, job, previous_time_period] <= model.y[machine, job, time_period] 
    else:
        return pyo.Constraint.Skip


def constraint_total_time(model, machine, job, time_period):
    return time_period * model.x[machine, job, time_period] <= model.C

In [14]:
model.add_component('constraint_unique_start', pyo.Constraint(model.machines, model.jobs, rule=constraint_unique_start))
model.add_component('constraint_unique_machine', pyo.Constraint(model.machines, model.time_periods, rule=constraint_unique_machine))
model.add_component('constraint_sequence_order', pyo.Constraint(model.sequences, rule=constraint_sequence_order))
model.add_component('constraint_job_duration', pyo.Constraint(model.machines, model.jobs, model.time_periods, rule=constraint_job_duration))
model.add_component('constraint_duration_complement', pyo.Constraint(model.machines, model.jobs, rule=constraint_duration_complement))
model.add_component('constraint_start_c1', pyo.Constraint(model.machines, model.jobs, model.time_periods, rule=constraint_start_c1))
model.add_component('constraint_start_c2', pyo.Constraint(model.machines, model.jobs, model.time_periods, rule=constraint_start_c2))
model.add_component('constraint_start_c3', pyo.Constraint(model.machines, model.jobs, model.time_periods, rule=constraint_start_c3))
model.add_component('constraint_total_time', pyo.Constraint(model.machines, model.jobs, model.time_periods, rule=constraint_total_time))

### Objective

In [15]:
model.add_component('objective_function', pyo.Objective(expr=model.C + 1, sense = pyo.minimize))

## Solve

In [16]:
solver = pyo.SolverFactory("appsi_highs")
solver.options["mip_heuristic_effort"] = 0.2
solver.options["time_limit"] = 120
solver.options["log_file"] = "highs.log"
solver.solve(model, tee=True)

Running HiGHS 1.7.2 (git hash: 184e327): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 2e+02]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 2e+01]
Presolving model
8696 rows, 3336 cols, 42741 nonzeros  0s
8686 rows, 3332 cols, 42673 nonzeros  0s

Solving MIP model with:
   8686 rows
   3332 cols (1591 binary, 0 integer, 0 implied int., 1741 continuous)
   42673 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   1               inf                  inf        0      0      0         0     0.7s
         0       0         0   0.00%   5.059816955     inf                  inf        0      0      4      5252     2.0s
         0       0         0   0.00%   5.17090552      inf                  in

RuntimeError: A feasible solution was not found, so no solution can be loaded. If using the appsi.solvers.Highs interface, you can set opt.config.load_solution=False. If using the environ.SolverFactory interface, you can set opt.solve(model, load_solutions = False). Then you can check results.termination_condition and results.best_feasible_objective before loading a solution.